In [7]:
import os
import torch
import torch.nn as nn
from torch.autograd import Variable
import torchvision.datasets as dset
from torch.utils.data import Dataset, DataLoader
from torchvision import utils
import torchvision.transforms as transforms
import numpy as np
import matplotlib.pyplot as plt
import torch.nn.functional as F
import torch.optim as optim
import pandas as pd

In [10]:
train = pd.read_csv('bplgen_train.csv')
train_tensor = torch.tensor(train.values)
print(train_tensor.size())

torch.Size([99999, 1226])


In [14]:
batch_size = 100

train_loader = torch.utils.data.DataLoader(
                 dataset=train_tensor,
                 batch_size=batch_size,
                 shuffle=True)
print ('==>>> total training batch number: {}'.format(len(train_loader)))

==>>> total trainning batch number: 1000


In [16]:
test = pd.read_csv('bplgen_test.csv')
test_tensor = torch.tensor(test.values)
print(test_tensor.size())

torch.Size([24999, 1226])


In [17]:
test_loader = torch.utils.data.DataLoader(
                 dataset=test_tensor,
                 batch_size=batch_size,
                 shuffle=True)
print ('==>>> total testing batch number: {}'.format(len(test_loader)))

==>>> total testing batch number: 250


In [18]:
## network
class MLPNet(nn.Module):
    def __init__(self):
        super(MLPNet, self).__init__()
        self.fc1 = nn.Linear(35*35, 500)
        self.fc2 = nn.Linear(500, 256)
        self.fc3 = nn.Linear(256, 50)
    def forward(self, x):
        x = x.view(-1, 35*35)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x
    
    def name(self):
        return "MLP"

In [19]:
class LeNet(nn.Module):
    def __init__(self):
        super(LeNet, self).__init__()
        self.conv1 = nn.Conv2d(1, 20, 5, 1)
        self.conv2 = nn.Conv2d(20, 50, 5, 1)
        self.fc1 = nn.Linear(4*4*50, 500)
        self.fc2 = nn.Linear(500, 10)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.max_pool2d(x, 2, 2)
        x = F.relu(self.conv2(x))
        x = F.max_pool2d(x, 2, 2)
        x = x.view(-1, 4*4*50)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x
    
    def name(self):
        return "LeNet"

In [20]:
model = LeNet()

In [21]:
criterion = nn.CrossEntropyLoss()

In [25]:
for epoch in range(10):
    # trainning
    ave_loss = 0
    for batch_idx, (x, target) in enumerate(train_loader):
        optimizer.zero_grad()
        if use_cuda:
            x, target = x.cuda(), target.cuda()
        x, target = Variable(x), Variable(target)
        out = model(x)
        loss = criterion(out, target)
        ave_loss = ave_loss * 0.9 + loss.data[0] * 0.1
        loss.backward()
        optimizer.step()
        if (batch_idx+1) % 100 == 0 or (batch_idx+1) == len(train_loader):
            print ('==>>> epoch: {}, batch index: {}, train loss: {:.6f}'.format(
                epoch, batch_idx+1, ave_loss))
    # testing
    correct_cnt, ave_loss = 0, 0
    total_cnt = 0
    for batch_idx, (x, target) in enumerate(test_loader):
        if use_cuda:
            x, target = x.cuda(), target.cuda()
        x, target = Variable(x, volatile=True), Variable(target, volatile=True)
        out = model(x)
        loss = criterion(out, target)
        _, pred_label = torch.max(out.data, 1)
        total_cnt += x.data.size()[0]
        correct_cnt += (pred_label == target.data).sum()
        # smooth average
        ave_loss = ave_loss * 0.9 + loss.data[0] * 0.1
        
        if(batch_idx+1) % 100 == 0 or (batch_idx+1) == len(test_loader):
            print ('==>>> epoch: {}, batch index: {}, test loss: {:.6f}, acc: {:.3f}'.format(
                epoch, batch_idx+1, ave_loss, correct_cnt * 1.0 / total_cnt))

ValueError: too many values to unpack (expected 2)

In [ ]:
torch.save(model.state_dict(), model.name())